## Downloading the Dataset

Dataset Source: https://www.kaggle.com/competitions/global-wheat-detection

Before attempting to download the dataset, you have to get your kaggle API key and your kaggle username and set it to the google colab secrets as "KAGGLE_USERNAME" and "KAGGLE_KEY".

In [ ]:
# Mount the Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set up Kaggle API authentication

import os
from google.colab import userdata
import zipfile

os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

# Import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize and authenticate Kaggle API
api = KaggleApi()
api.authenticate()

print("Kaggle API authenticated successfully!")

In [ ]:
# Ensure dataset is downloaded via Kaggle API
!kaggle competitions download -c global-wheat-detection -p /content/

# Extract the dataset and verify extraction

import zipfile

# Path to the downloaded dataset
zip_path = '/content/global-wheat-detection.zip'
extract_to = '/content/global-wheat-detection'

# Extract the dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Dataset extracted to: {extract_to}")

# Faster R-CNN model without Augmentation

## Data Preparation

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")

In [ ]:
import pandas as pd
import torch
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import train_test_split

# Load the train.csv file
csv_file = '/content/global-wheat-detection/train.csv'
data = pd.read_csv(csv_file)

# Convert COCO bounding box format to Faster R-CNN format
def coco_to_frcnn(bbox):
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height
    return [x_min, y_min, x_max, y_max]

# Parse bounding boxes from the dataframe
# Assuming `bbox` column contains strings in the format: "[x_min, y_min, width, height]"
data['bbox'] = data['bbox'].apply(eval)                                         # Convert string to list
data['bbox_frcnn'] = data['bbox'].apply(coco_to_frcnn)

# Group data by image_id to collect all bounding boxes for each image
annotations = {}
for image_id, group in data.groupby('image_id'):
    boxes = group['bbox_frcnn'].tolist()
    annotations[image_id] = {
        "boxes": torch.tensor(boxes, dtype=torch.float32),
        "labels": torch.tensor([1] * len(boxes), dtype=torch.int64)             # Assuming 1 class for wheat
    }

# Split data into train and validation sets
image_ids = list(annotations.keys())
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

# Dataset class
class WheatDataset(torch.utils.data.Dataset):
    def __init__(self, image_ids, annotations, image_dir, transforms=None):
        self.image_ids = image_ids
        self.annotations = annotations
        self.image_dir = image_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_path = f"{self.image_dir}/{image_id}.jpg"
        image = Image.open(image_path).convert("RGB")

        target = self.annotations[image_id]

        if self.transforms:
            image = self.transforms(image)

        return image, target

# Paths to images
data_dir = '/content/global-wheat-detection/train'

# Define datasets
train_dataset = WheatDataset(train_ids, annotations, data_dir, transforms=F.to_tensor)
val_dataset = WheatDataset(val_ids, annotations, data_dir, transforms=F.to_tensor)

# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

# Verify data loading
if __name__ == "__main__":
    for images, targets in train_loader:
        print("Images:", images)
        print("Targets:", targets)
        break

## First Dry Run (SGD as optimizer)

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision.models.detection.rpn import AnchorGenerator
import time
import os
import torchvision.ops as ops

# Define and load the model
# Ensure compatibility for both Google Colab and Kaggle
if "google.colab" in str(get_ipython()):
    save_dir = "/content/global-wheat-detection/FirstModel"
else:
    save_dir = "/kaggle/working/FirstModel"

os.makedirs(save_dir, exist_ok=True)

def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2                                                             # 1 class (wheat) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

model = get_model()

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer and learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

# IoU computation
def compute_iou(pred_boxes, gt_boxes):
    if len(pred_boxes) == 0 or len(gt_boxes) == 0:
        return 0.0                                                              # No detections or ground truth
    iou = ops.box_iou(pred_boxes, gt_boxes)
    return iou.mean().item()

# Training loop
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0.0
    for batch_idx, (images, targets) in enumerate(data_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        batch_loss = losses.item()
        total_loss += batch_loss

        print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}/{len(data_loader)}, Loss: {batch_loss:.4f}")

    average_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1} Training Loss: {average_loss:.4f}")
    return average_loss

# Validation loop
def evaluate(model, data_loader, device):
    model.eval()
    total_iou = 0.0
    num_batches = 0
    print("Running validation...")
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(data_loader):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            predictions = model(images)
            batch_iou = 0.0
            for pred, target in zip(predictions, targets):
                pred_boxes = pred["boxes"].detach().cpu()
                gt_boxes = target["boxes"].detach().cpu()
                batch_iou += compute_iou(pred_boxes, gt_boxes)

            avg_batch_iou = batch_iou / len(images) if len(images) > 0 else 0
            total_iou += avg_batch_iou
            num_batches += 1

            print(f"Validation Batch {batch_idx + 1}/{len(data_loader)}, IoU: {avg_batch_iou:.4f}")

    avg_iou = total_iou / num_batches if num_batches > 0 else None
    print(f"Validation completed. Avg IoU: {avg_iou:.4f}")
    return None, avg_iou                                                        # No validation loss

# Dictionary to store results
results = {'epoch': [], 'train_loss': [], 'val_loss': [], 'val_iou': []}

# Train the model
num_epochs = 10                                                                 # Number of epochs
for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch + 1}/{num_epochs}")
    start_time = time.time()

    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    print("Evaluating on validation set...")
    _, val_iou = evaluate(model, val_loader, device)                            # No validation loss

    results['epoch'].append(epoch + 1)
    results['train_loss'].append(train_loss)
    results['val_loss'].append(None)                                            # No validation loss
    results['val_iou'].append(val_iou)

    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(save_dir, 'training_results.csv'), index=False)

    checkpoint_path = os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth')
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Saved model checkpoint to {checkpoint_path}")

    end_time = time.time()
    print(f"Epoch {epoch + 1} completed in {(end_time - start_time):.2f} seconds")
    print(f"Epoch {epoch + 1} Summary: Train Loss: {train_loss:.4f}, Validation IoU: {val_iou:.4f}\n")

# Plot results
def plot_results(results_csv):
    import matplotlib.pyplot as plt
    results_df = pd.read_csv(results_csv)

    plt.figure(figsize=(8,6))
    plt.plot(results_df['epoch'], results_df['train_loss'], label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(8,6))
    plt.plot(results_df['epoch'], results_df['val_iou'], label='Validation IoU', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.title('Validation IoU Over Epochs')
    plt.legend()
    plt.show()

plot_results(os.path.join(save_dir, 'training_results.csv'))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torch
from PIL import Image
import torchvision.transforms as T
import os

# Load the trained model
model_path = os.path.join(save_dir, f'model_epoch_{num_epochs}.pth')            # Load the last epoch model
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Define image path
test_image_id = "f5a1f0358"                                                     # Update with actual test image ID

# Handle path for both Colab and Kaggle
if "google.colab" in str(get_ipython()):
    test_dir = "/content/global-wheat-detection/data/test"
else:
    test_dir = "/kaggle/input/global-wheat-detection/test"

# Get test images
test_image_path = os.path.join(test_dir, f"{test_image_id}.jpg")

# Load image
image = Image.open(test_image_path).convert("RGB")
transform = T.ToTensor()
image_tensor = transform(image).unsqueeze(0).to(device)

# Get model predictions
with torch.no_grad():
    predictions = model(image_tensor)

# Extract predicted boxes & scores
pred_boxes = predictions[0]["boxes"].cpu().numpy()
scores = predictions[0]["scores"].cpu().numpy()

# Set confidence threshold
confidence_threshold = 0.5
filtered_boxes = pred_boxes[scores > confidence_threshold]

# Plot the image
fig, ax = plt.subplots(1, figsize=(8, 6))
ax.imshow(image)


# Draw bounding boxes
for box in filtered_boxes:
    x_min, y_min, x_max, y_max = box
    rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor="r", facecolor="none")
    ax.add_patch(rect)

plt.title("Predicted Bounding Boxes")
plt.show()

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision.models.detection.rpn import AnchorGenerator
import time
import os
import torchvision.ops as ops
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T

# Define the model
def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2  # 1 class (wheat) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

model = get_model()

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Load trained model
model_path = os.path.join(save_dir, f'model_epoch_{num_epochs}.pth')
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# Function to visualize bounding boxes with labels
def visualize_predictions(image_path, model, device, confidence_threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        predictions = model(image_tensor)

    pred_boxes = predictions[0]["boxes"].cpu().numpy()
    scores = predictions[0]["scores"].cpu().numpy()

    filtered_boxes = pred_boxes[scores > confidence_threshold]
    filtered_scores = scores[scores > confidence_threshold]

    fig, ax = plt.subplots(1, figsize=(8, 6))
    ax.imshow(image)

    for box, score in zip(filtered_boxes, filtered_scores):
        x_min, y_min, x_max, y_max = box
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor="r", facecolor="none")
        ax.add_patch(rect)
        ax.text(x_min, y_min - 5, f'{score:.2f}', color='red', fontsize=10, weight='bold', bbox=dict(facecolor='white', alpha=0.5))

    image_name = os.path.basename(image_path)
    plt.title(f"Predicted Bounding Boxes - {image_name}")
    plt.show()

# Define test image path for Kaggle
if "google.colab" in str(get_ipython()):
    test_dir = "/content/global-wheat-detection/data/test"
else:
    test_dir = "/kaggle/input/global-wheat-detection/test"

# Get test image
test_image_path = os.path.join(test_dir, f"{test_image_id}.jpg")

# Visualize bounding boxes for up to 10 test images
for img_path in test_images[:10]:
    visualize_predictions(img_path, model, device)

---

## Second Dry Run (Adam as optimizer and learning rate scheduler)

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision.models.detection.rpn import AnchorGenerator
import time
import os
import torchvision.ops as ops
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T

# Define and load the model
# Ensure compatibility for both Google Colab and Kaggle
if "google.colab" in str(get_ipython()):
    save_dir = "/content/global-wheat-detection/SecondModel"
else:
    save_dir = "/kaggle/working/SecondModel"

os.makedirs(save_dir, exist_ok=True)

def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2                                                             # 1 class (wheat) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

model = get_model()

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Define optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)

# Learning rate scheduler - Reduce LR when validation IoU plateaus
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, verbose=True)

# IoU computation
def compute_iou(pred_boxes, gt_boxes):
    if len(pred_boxes) == 0 or len(gt_boxes) == 0:
        return 0.0                                                              # No detections or ground truth
    iou = ops.box_iou(pred_boxes, gt_boxes)
    return iou.mean().item()

# Training loop
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0.0
    for batch_idx, (images, targets) in enumerate(data_loader):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        batch_loss = losses.item()
        total_loss += batch_loss

        print(f"Epoch {epoch + 1}, Batch {batch_idx + 1}/{len(data_loader)}, Loss: {batch_loss:.4f}")

    average_loss = total_loss / len(data_loader)
    print(f"Epoch {epoch + 1} Training Loss: {average_loss:.4f}")
    return average_loss

# Validation loop
def evaluate(model, data_loader, device):
    model.eval()
    total_iou = 0.0
    num_batches = 0
    print("Running validation...")
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(data_loader):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            predictions = model(images)
            batch_iou = 0.0
            for pred, target in zip(predictions, targets):
                pred_boxes = pred["boxes"].detach().cpu()
                gt_boxes = target["boxes"].detach().cpu()
                batch_iou += compute_iou(pred_boxes, gt_boxes)

            avg_batch_iou = batch_iou / len(images) if len(images) > 0 else 0
            total_iou += avg_batch_iou
            num_batches += 1

            print(f"Validation Batch {batch_idx + 1}/{len(data_loader)}, IoU: {avg_batch_iou:.4f}")

    avg_iou = total_iou / num_batches if num_batches > 0 else None
    print(f"Validation completed. Avg IoU: {avg_iou:.4f}")
    return None, avg_iou                                                        # No validation loss

# Dictionary to store results
results = {'epoch': [], 'train_loss': [], 'val_loss': [], 'val_iou': []}

# Train the model
num_epochs = 10                                                                 # Number of epochs
for epoch in range(num_epochs):
    print(f"Starting Epoch {epoch + 1}/{num_epochs}")
    start_time = time.time()

    train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
    print("Evaluating on validation set...")
    _, val_iou = evaluate(model, val_loader, device)                            # No validation loss

    results['epoch'].append(epoch + 1)
    results['train_loss'].append(train_loss)
    results['val_loss'].append(None)                                            # No validation loss
    results['val_iou'].append(val_iou)

    # Apply learning rate scheduler
    lr_scheduler.step(val_iou)                                                  # Adjust learning rate based on validation IoU

    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(save_dir, 'training_results.csv'), index=False)

    checkpoint_path = os.path.join(save_dir, f'model_epoch_{epoch + 1}.pth')
    torch.save(model.state_dict(), checkpoint_path)
    print(f"Saved model checkpoint to {checkpoint_path}")

    end_time = time.time()
    print(f"Epoch {epoch + 1} completed in {(end_time - start_time):.2f} seconds")
    print(f"Epoch {epoch + 1} Summary: Train Loss: {train_loss:.4f}, Validation IoU: {val_iou:.4f}\n")

# Plot results
def plot_results(results_csv):
    import matplotlib.pyplot as plt
    results_df = pd.read_csv(results_csv)

    plt.figure(figsize=(8,6))
    plt.plot(results_df['epoch'], results_df['train_loss'], label='Train Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.show()

    plt.figure(figsize=(8,6))
    plt.plot(results_df['epoch'], results_df['val_iou'], label='Validation IoU', color='green')
    plt.xlabel('Epoch')
    plt.ylabel('IoU')
    plt.title('Validation IoU Over Epochs')
    plt.legend()
    plt.show()

plot_results(os.path.join(save_dir, 'training_results.csv'))

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision.models.detection.rpn import AnchorGenerator
import time
import os
import torchvision.ops as ops
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T

# Define and load the model
# Ensure compatibility for both Google Colab and Kaggle
if "google.colab" in str(get_ipython()):
    save_dir = "/content/global-wheat-detection/SecondModel"
else:
    save_dir = "/kaggle/working/SecondModel"

os.makedirs(save_dir, exist_ok=True)

def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2                                                             # 1 class (wheat) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

model = get_model()

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Load last trained model checkpoint
training_results_path = os.path.join(save_dir, 'training_results.csv')
if os.path.exists(training_results_path):
    results_df = pd.read_csv(training_results_path)
    last_completed_epoch = results_df['epoch'].max()
    print(f"Last completed epoch: {last_completed_epoch}")
else:
    raise FileNotFoundError("No training results found. Make sure training has been completed before visualization.")

model_path = os.path.join(save_dir, f'model_epoch_{last_completed_epoch}.pth')
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print(f"Loaded model from epoch {last_completed_epoch}")
else:
    raise FileNotFoundError(f"Model checkpoint for epoch {last_completed_epoch} not found.")

 Function to visualize bounding boxes with labels and image name
def visualize_predictions(image_path, model, device, confidence_threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        predictions = model(image_tensor)

    pred_boxes = predictions[0]["boxes"].cpu().numpy()
    scores = predictions[0]["scores"].cpu().numpy()

    filtered_boxes = pred_boxes[scores > confidence_threshold]
    filtered_scores = scores[scores > confidence_threshold]

    fig, ax = plt.subplots(1, figsize=(8, 6))
    ax.imshow(image)

    for box, score in zip(filtered_boxes, filtered_scores):
        x_min, y_min, x_max, y_max = box
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor="r", facecolor="none")
        ax.add_patch(rect)
        ax.text(x_min, y_min - 5, f'{score:.2f}', color='red', fontsize=10, weight='bold', bbox=dict(facecolor='white', alpha=0.5))

    image_name = os.path.basename(image_path)
    plt.title(f"Predicted Bounding Boxes - {image_name}")
    plt.show()

# Load test images and visualize
if "google.colab" in str(get_ipython()):
    test_dir = "/content/global-wheat-detection/data/test"
else:
    test_dir = "/kaggle/input/global-wheat-detection/test"

test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith(".jpg")]

# Visualize bounding boxes for up to 10 test images
for img_path in test_images[:10]:
    visualize_predictions(img_path, model, device)


---

## Third Dry Run - K-Fold Cross-Validation (New Data Preparation applied)

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import os
import time
import torchvision.ops as ops
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision.transforms as T

# Load Dataset with K-Fold Cross-Validation
# Ensure compatibility for both Google Colab and Kaggle
if "google.colab" in str(get_ipython()):
    save_dir = "/content/global-wheat-detection/ThirdModel"
    csv_file = "/content/global-wheat-detection/data/train.csv"
    data_dir = "/content/global-wheat-detection/data/train"
else:
    save_dir = "/kaggle/working/ThirdModel"
    csv_file = "/kaggle/input/global-wheat-detection/train.csv"
    data_dir = "/kaggle/input/global-wheat-detection/train"

os.makedirs(save_dir, exist_ok=True)

data = pd.read_csv(csv_file)

def coco_to_frcnn(bbox):
    x_min, y_min, width, height = bbox
    x_max = x_min + width
    y_max = y_min + height
    return [x_min, y_min, x_max, y_max]

data['bbox'] = data['bbox'].apply(eval)
data['bbox_frcnn'] = data['bbox'].apply(coco_to_frcnn)
annotations = {}
for image_id, group in data.groupby('image_id'):
    boxes = group['bbox_frcnn'].tolist()
    annotations[image_id] = {
        "boxes": torch.tensor(boxes, dtype=torch.float32),
        "labels": torch.tensor([1] * len(boxes), dtype=torch.int64)
    }

image_ids = list(annotations.keys())

class WheatDataset(torch.utils.data.Dataset):
    def __init__(self, image_ids, annotations, image_dir, transforms=None):
        self.image_ids = image_ids
        self.annotations = annotations
        self.image_dir = image_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_path = f"{self.image_dir}/{image_id}.jpg"
        image = Image.open(image_path).convert("RGB")
        target = self.annotations[image_id]

        if self.transforms:
            image = self.transforms(image)

        return image, target

dataset = WheatDataset(image_ids, annotations, data_dir, transforms=F.to_tensor)
print("Dataset loaded successfully! Total images:", len(dataset))

In [ ]:
# Define and load the model
def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2                                                             # 1 class (wheat) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Dictionary to store results
results = {'fold': [], 'epoch': [], 'train_loss': [], 'val_iou': []}

# Function to print current learning rate
def print_lr(optimizer):
    for param_group in optimizer.param_groups:
        print(f"Current Learning Rate: {param_group['lr']:.6f}")

# IoU computation
def compute_iou(pred_boxes, gt_boxes):
    if len(pred_boxes) == 0 or len(gt_boxes) == 0:
        return 0.0  # No detections or ground truth
    iou = ops.box_iou(pred_boxes, gt_boxes)
    return iou.mean().item()

# Training loop
def train_one_epoch(model, optimizer, data_loader, device, epoch, total_epochs, fold):
    model.train()
    total_loss = 0.0
    start_epoch_time = time.time()
    for batch_idx, (images, targets) in enumerate(data_loader):
        start_time = time.time()
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        total_loss += losses.item()
        end_time = time.time()
        print(f"Fold {fold + 1}/{k_folds}, Epoch {epoch + 1}/{total_epochs}, Batch {batch_idx + 1}/{len(data_loader)}, "
              f"Loss: {losses.item():.4f}, Time: {end_time - start_time:.2f}s, ")
    end_epoch_time = time.time()
    print(f"Epoch {epoch + 1} completed in {(end_epoch_time - start_epoch_time)/60:.2f} min")
    return total_loss / len(data_loader)

# Validation loop
def evaluate(model, data_loader, device):
    model.eval()
    total_iou = 0.0
    num_batches = 0
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(data_loader):
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            predictions = model(images)
            batch_iou = 0.0
            for pred, target in zip(predictions, targets):
                pred_boxes = pred["boxes"].detach().cpu()
                gt_boxes = target["boxes"].detach().cpu()
                batch_iou += compute_iou(pred_boxes, gt_boxes)

            total_iou += batch_iou / len(images) if len(images) > 0 else 0
            num_batches += 1
            print(f"Validation Batch {batch_idx + 1}/{len(data_loader)}, IoU: {batch_iou:.4f}")

    return total_iou / num_batches if num_batches > 0 else 0

# K-Fold Cross Validation
k_folds = 3
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
indices = list(range(len(dataset)))
batch_size = 4

for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
    print(f"Starting Fold {fold + 1}/{k_folds}")

    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

    model = get_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.0001)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

    for epoch in range(3):
        print(f"Fold {fold + 1}, Epoch {epoch + 1}")
        train_loss = train_one_epoch(model, optimizer, train_loader, device, epoch, 3, fold)
        val_iou = evaluate(model, val_loader, device)

        results['fold'].append(fold + 1)
        results['epoch'].append(epoch + 1)
        results['train_loss'].append(train_loss)
        results['val_iou'].append(val_iou)

        lr_scheduler.step(val_iou)
        print_lr(optimizer)

        # Save model checkpoint
        checkpoint_path = os.path.join(save_dir, f'model_fold_{fold + 1}_epoch_{epoch + 1}.pth')
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Model checkpoint saved at {checkpoint_path}")

        print(f"Fold {fold + 1}, Epoch {epoch + 1} Summary: Train Loss: {train_loss:.4f}, Validation IoU: {val_iou:.4f}")

In [ ]:
import pandas as pd
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from PIL import Image
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import os
import time
import torchvision.ops as ops
import torchvision.transforms as T
import matplotlib.patches as patches
import matplotlib.pyplot as plt

# Ensure compatibility for both Google Colab and Kaggle
if "google.colab" in str(get_ipython()):
    save_dir = "/content/global-wheat-detection/ThirdModel"
    test_dir = "/content/global-wheat-detection/data/test"
else:
    save_dir = "/kaggle/working/ThirdModel"
    test_dir = "/kaggle/input/global-wheat-detection/test"

os.makedirs(save_dir, exist_ok=True)

# Define and load the model
def get_model():
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    num_classes = 2  # 1 class (wheat) + background
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
    return model

# Move model to device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Function to load trained model
def load_model(fold_num, epoch_num):
    model_path = os.path.join(save_dir, f"model_fold_{fold_num}_epoch_{epoch_num}.pth")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model checkpoint not found: {model_path}")

    model = get_model()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    print(f"Loaded model from {model_path}")
    return model

# Function to visualize predictions
def visualize_predictions(image_path, model, device, confidence_threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    transform = T.ToTensor()
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        predictions = model(image_tensor)

    pred_boxes = predictions[0]["boxes"].cpu().numpy()
    scores = predictions[0]["scores"].cpu().numpy()

    # Filter predictions by confidence threshold
    filtered_boxes = pred_boxes[scores > confidence_threshold]
    filtered_scores = scores[scores > confidence_threshold]

    # Plot image with bounding boxes
    fig, ax = plt.subplots(1, figsize=(8, 6))
    ax.imshow(image)

    for box, score in zip(filtered_boxes, filtered_scores):
        x_min, y_min, x_max, y_max = box
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor="r", facecolor="none")
        ax.add_patch(rect)
        ax.text(x_min, y_min - 5, f"{score:.2f}", color="red", fontsize=10, weight="bold")

    plt.title(f"{os.path.basename(image_path)}")
    plt.show()

# Directory containing test images
test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith(".jpg")]

# Load the best model based on the best-performing epoch
csv_path = os.path.join(save_dir, 'training_results.csv')
df = pd.read_csv(csv_path)
best_epoch = df["epoch"].max()  # Get the best epoch
best_fold = df["fold"].max()  # Get the best fold

model = load_model(best_fold, best_epoch)

# Visualize predictions for 10 test images
for img_path in test_images[:10]:
    visualize_predictions(img_path, model, device)